## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-20-20-52-27 +0000,bbc,Hurricane Erin's high waves thrash coast of No...,https://www.bbc.com/news/articles/ce934e9gyryo...
1,2025-08-20-20-51-16 +0000,nypost,Official say US military raid in Syria killed ...,https://nypost.com/2025/08/20/us-news/official...
2,2025-08-20-20-50-28 +0000,nyt,"Stephanie Shirley, Who Created a Tech World fo...",https://www.nytimes.com/2025/08/20/business/st...
3,2025-08-20-20-47-16 +0000,nyt,Camp Mystic Parents Demand Changes Before More...,https://www.nytimes.com/2025/08/20/us/politics...
4,2025-08-20-20-46-02 +0000,nypost,Pop Mart shares surge 12% after CEO says mini ...,https://nypost.com/2025/08/20/business/pop-mar...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2329/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
64,trump,42
71,new,20
77,ukraine,12
592,up,12
219,he,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
9,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...,113
306,2025-08-19-22-54-43 +0000,nypost,Russia launches largest strike on Ukraine in w...,https://nypost.com/2025/08/19/world-news/russi...,93
52,2025-08-20-19-20-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...,88
44,2025-08-20-19-38-44 +0000,nyt,Trump Calls for Fed Governor Lisa Cook to Resi...,https://www.nytimes.com/2025/08/20/us/politics...,85
137,2025-08-20-15-35-01 +0000,nypost,White House trolls Mark Hamill over claim he n...,https://nypost.com/2025/08/20/us-news/white-ho...,81


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
9,113,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...
25,49,2025-08-20-20-14-32 +0000,nypost,"140-pound, 12-foot python captured near South ...",https://nypost.com/2025/08/20/us-news/12-foot-...
314,49,2025-08-19-21-56-00 +0000,wsj,"Flight Attendants Win Boarding Pay, Four-Year ...",https://www.wsj.com/business/airlines/air-cana...
52,46,2025-08-20-19-20-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...
60,44,2025-08-20-18-56-00 +0000,wsj,Fed minutes from the July meeting reveal broad...,https://www.wsj.com/economy/central-banking/fe...
78,44,2025-08-20-18-09-26 +0000,nypost,"Israel calls up 60,000 reservists for looming ...",https://nypost.com/2025/08/20/world-news/israe...
273,38,2025-08-20-01-29-42 +0000,cbc,Kristi Noem says U.S. will paint Mexico border...,https://www.cbc.ca/news/world/noem-us-border-w...
193,35,2025-08-20-10-00-00 +0000,latimes,Barabak: Power grab may energize Newsom and De...,https://www.latimes.com/politics/story/2025-08...
157,35,2025-08-20-14-01-00 +0000,wsj,Billionaire Bill Ackman has a new fascination:...,https://www.wsj.com/us-news/education/bill-ack...
183,32,2025-08-20-11-59-04 +0000,nypost,Martha Nolan-O’Slatarra’s boyfriend gives hear...,https://nypost.com/2025/08/20/us-news/martha-n...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
